In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Mon Jun 22 13:30:04 PDT 2020


In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/25draw_5seed/india/2020_06_19_18_11_00/count_data/']

locations = ['India']

In [3]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,21,iron_fortification_scale_up,0.0
4,2020,1_to_4,female,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0


In [4]:
np.unique(pt['folic_acid_fortification_group'])

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [5]:
np.unique(pt['vitamin_a_fortification_group'])

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

# Vitamin A Fortification Coverage

In [6]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','vitamin_a_fortification_scale_up'])] # look only at baseline and vitamin a scenario
covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

overall = data.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / overall
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                         year                                        
baseline                         2020   25.0  0.242876  0.006767  0.229482   
                                 2021   25.0  0.242922  0.006945  0.229136   
                                 2022   25.0  0.243046  0.006839  0.229524   
                                 2023   25.0  0.243002  0.006905  0.229145   
                                 2024   25.0  0.243095  0.006949  0.229407   
                                 2025   25.0  0.242852  0.007168  0.230200   
vitamin_a_fortification_scale_up 2020   25.0  0.806627  0.004194  0.797432   
                                 2021   25.0  0.898241  0.003843  0.889715   
                                 2022   25.0  0.922278  0.003669  0.913985   
                                 2023   25.0  0.926665  0.003173  0.919040   
                                 2024   25.0  0.929339  0.003094  0.922569   
                                 2025   25.0  0.930334  0.003379  0.923621   

                                            25%       50%       75%       max  
scenario                         year                                          
baseline                         2020  0.236546  0.244814  0.247557  0.252775  
                                 2021  0.236734  0.244224  0.247569  0.254729  
                                 2022  0.237280  0.244273  0.247161  0.255962  
                                 2023  0.237245  0.243118  0.247017  0.255388  
                                 2024  0.236975  0.244230  0.247715  0.254922  
                                 2025  0.237210  0.243044  0.247549  0.256353  
vitamin_a_fortification_scale_up 2020  0.803766  0.807786  0.809862  0.812552  
                                 2021  0.895452  0.899367  0.901065  0.903130  
                                 2022  0.919109  0.923201  0.925418  0.927368  
                                 2023  0.924087  0.927227  0.929254  0.931600  
                                 2024  0.926810  0.929561  0.931696  0.934185  
                                 2025  0.928247  0.930377  0.932613  0.936196

## Vitamin A coverage conclusion


- coverage is starting in 2020 still, not 2021, although the coverage value that intervention scenario doesn't make sense to me
- based on the coverage algorithm, the 2021 coverage in the intervention scenario should be 89%, which it is.
- not sure why 2020 intervention scenario coverage in 2020 is 80%... this is outside of the confidence interval for parameter b in the coverage algorithm

# Folic Acid Fortification Coverage

- unexpected results when including folic_acid_fortification_group == 'unknown' values
- try again by excluding unknown coverage...

In [9]:
# COMPUTE COVERAGE PROPORTION BY SCENARIO, YEAR - Folic acid

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','folic_acid_fortification_scale_up'])]
covered = data.loc[data.folic_acid_fortification_group == 'covered']
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

known_coverage = data.loc[data.folic_acid_fortification_group != 'unknown']
known_coverage = known_coverage.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / known_coverage
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                          year                                        
baseline                          2020   25.0  0.063375  0.005361  0.054098   
                                  2021   25.0  0.063282  0.004275  0.057301   
                                  2022   25.0  0.062937  0.004420  0.056931   
                                  2023   25.0  0.062906  0.004635  0.055450   
                                  2024   25.0  0.062964  0.004615  0.055594   
                                  2025   25.0  0.062848  0.004583  0.055094   
folic_acid_fortification_scale_up 2020   25.0  0.063221  0.005121  0.054106   
                                  2021   25.0  0.063336  0.004225  0.057301   
                                  2022   25.0  0.064335  0.004398  0.058818   
                                  2023   25.0  0.079515  0.004530  0.073162   
                                  2024   25.0  0.105076  0.005014  0.098150   
                                  2025   25.0  0.125640  0.005256  0.117178   

                                             25%       50%       75%       max  
scenario                          year                                          
baseline                          2020  0.060439  0.062261  0.066634  0.075846  
                                  2021  0.060694  0.061869  0.066238  0.072205  
                                  2022  0.059560  0.061787  0.065986  0.070938  
                                  2023  0.059198  0.062659  0.067293  0.070325  
                                  2024  0.058806  0.062980  0.067028  0.070276  
                                  2025  0.058192  0.062924  0.066366  0.069956  
folic_acid_fortification_scale_up 2020  0.060140  0.063255  0.065040  0.075846  
                                  2021  0.060150  0.062663  0.065231  0.072205  
                                  2022  0.060597  0.063364  0.066470  0.072221  
                                  2023  0.074656  0.079567  0.082914  0.087334  
                                  2024  0.099397  0.106007  0.108898  0.113191  
                                  2025  0.121024  0.126572  0.129703  0.134130

## Folic Acid Fortification Coverage Conclusion
- Looks as expected based on coverage algorithm, although differences are slightly hard to discern due to very close values for parameters a and b in algorithm
